# Aroma Notebook for Colab

## Config

In [ ]:
#@title Configurations
#@markdown https://dashboard.ngrok.com/get-started/your-authtoken for ngrok token

repo_url = 'https://github.com/lumiknit/aroma' #@param {type:"string"}
hf_model_repo = 'lumiknit/diffusers-model-archive' #@param {type:"string"}
webui_username = 'asdf' #@param {type:"string"}
webui_password = '1q2w3e4r' #@param {type:"string"}
daemon_password = 'aroma1234' #@param {type:"string"}
ngrok_token = '-' #@param {type:"string"}

## Setup

In [ ]:
!rm -rf /ws

In [ ]:
!curl -sL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!apt-get update && apt-get install -y jq nodejs
!git lfs install
!pip install pyngrok pytorch-lightning numpy pillow scipy ftfy transformers diffusers==0.16.1 accelerate==0.18.0 safetensors omegaconf huggingface_hub

In [ ]:
# Clone repo and setup
%cd /
!git clone $repo_url /ws/aroma
%cd /ws/aroma
!git lfs install
!mkdir -p /ws/aroma/models /ws/aroma/archives /ws/aroma/outputs /ws/aroma/state

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id=hf_model_repo, local_dir="/ws/aroma/models/a")

In [ ]:
# Change host and port to 0.0.0.0:9125
import json

config = {
    'webui': {
        'host': '0.0.0.0',
        'port': 9125,
        'username': webui_username,
        'password': webui_password,
    },
    'save_raw': False,
    'password': daemon_password,
    'init_values': {
        'model': {
            'path': "a/anything-v4.5"
        },
        'textual_inversions': [
            #"a/textual_inversions/EasyNegative.safetensors",
            "a/textual_inversions/EasyNegativeV2.safetensors",
            #"a/textual_inversions/bad-picture-chill-75v.pt",
            #"a/textual_inversions/ng_deepnegative_v1_75t.pt",
            #"a/textual_inversions/realisticvision-negative-embedding.pt",
            #"a/textual_inversions/verybadimagenegative_v1.3.pt",
        ]
    }
}
config_json = json.dumps(config)

!echo '{config_json}' > /ws/aroma/config.json
!cp /ws/aroma/daemon/run_without_activate.sh /ws/aroma/daemon/run.sh
!touch /ws/aroma/state/values.json

In [ ]:
# Install node packages
!cd /ws/aroma/ui && npm install -g npm && npm install
!node --version

# Run

In [ ]:
import os
import threading
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9125, auth=f"{webui_username}:{webui_password}").public_url
print(f"{public_url}")
!node ui/main.js